# Bead center detection algorithm

This notebook describes the center detection algorithm used to identify the center of the tantalum beads. Identifying the center can be challenging as the bead is approx. 10 pixels wide on EOS images.


## Problem

The problem is to identify the peak of the images below. We split the problem in two:

1. Getting close to the center
1. Fine-tuning the center

Each problem is explored separately below.


In [ ]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
from matplotlib import patches
from pathlib import Path
from scipy.optimize import minimize, brute
from sklearn import metrics
from scipy import stats
import math
import pandas as pd
from numba import jit
%matplotlib inline


## Getting close to the center

For now we have no useful way to identify the peaks. Thus the only way to get close to the bead centers (peaks) is to have the user click it, and we asume the user's click is within a few pixel of the true center. The image below shows a crop of a scan with the beads visible as green dots. We then pass the crop in the black square on to the fine-tuning algorithm.

In [ ]:
img_global = cv2.imread("../data/DX000065 top beads.tif", cv2.IMREAD_UNCHANGED)
P0_global = np.array([23, 179])

#img_global = cv2.imread("../data/DX000082 close beads.tif", cv2.IMREAD_UNCHANGED)
#P0_global = np.array([42, 34])

assert img_global is not None, "Image not loaded"

plt.figure(figsize=(14,8))
plt.title("Raw image with beads")

plt.imshow(img_global)
plt.colorbar().set_label("pixel value")

cropbox_canvas = np.array([15, 15])
cropbox_offset = (P0_global - cropbox_canvas/2).astype(int)
cropbox = patches.Rectangle(
    cropbox_offset, 
    *cropbox_canvas.tolist(),
    fill=False
)
plt.gca().add_patch(cropbox)

plt.show()

## Fine-tuning the center

We aim to identify the exact peak with as high precision as achievable within a reasonable processing time.

The approach can be broken down into the following steps:

1. Select an appropriate origin, here we chose (8,7).
1. Convert the image from cartesian to polar coordinates* and drop the angle to give a 1d image of radii from the given origin.
1. Find parameters for the sinc function that minimizes MSE to the given polar radius-only image.
1. Select new origin and repeat from step 2 until until convergence.

\*the implementation only calculates radii and not angle.

In [ ]:
plt.figure(figsize=(14,6))
plt.subplot(121)
plt.title("Raw crop of bead")
img_raw = img_global[
    cropbox_offset[1]:cropbox_offset[1]+cropbox_canvas[1],
    cropbox_offset[0]:cropbox_offset[0]+cropbox_canvas[0]
]

imgobj1 = plt.imshow(img_raw)
plt.colorbar(imgobj1)

plt.subplot(122)
plt.title("Blurred using Gaussian (7,7) kernel, normalized")
img = cv2.GaussianBlur(img_raw, (7,7), 0)
#img = cv2.blur(img_raw, (7,7))
#img = img - np.min(img)
#img = img / np.max(img)
imgobj2 = plt.imshow(img)
plt.colorbar(imgobj2).set_label("pixel value")

plt.show()

### Problem details

In the image above, it is easy to see that the center is around (8,7). We will be looking for this center in two steps:

1. Brute force (grid search) to identify the location of the peak, each pixel within a few pixels' radius is searched.
1. Local gradient descent of a cost function that takes (x,y) as parameters

### Converting image to polar coordinates

We convert the image from cartesian to polar coordinates and drop the angle dimension to effectively give is a 1-dimensional image with index vector $R$ and value vector $V$ for the given cartesian origin $(x,y)$ in the cropped image. Normalise $V$, then plot $V$ against $R$ for $R \leq 2.0$.

In [ ]:
def reduce_img_to_polar_radius(img: np.ndarray, x: float, y: float, **kwargs):
    """
    Convert a 2d image in cartesian coordinates to polar coordinates and drop angle.
    
    This function convert the image from 2d cartesian to 1d radius-only polar image.
    
    Parameters
    ----------
    img: ndarray
        Image to convert
    x: float
        x coordinate to use as origin for cartesian to polar conversion
    y: float
        y coordinate to use as origin for cartesian to polar conversion
    r_max: float
        Drop pixels with a radius larger than this value, optional
    normalize: boolean
        Return normalized image, e.g. all values are between 0 and 1
        
    Returns
    -------
    ndarray of shape (2,) where (0,x) describes radius and (1,x) the corresponding pixel value
    """
    X, Y = np.meshgrid(
        np.arange(img.shape[1]),
        np.arange(img.shape[0])
    )
    R = np.sqrt(
        np.square(X - x) + 
        np.square(Y - y)
    )
    
    V = np.copy(img)

    if kwargs.get("r_max", 0.0) > 0.0:
        Xr, Yr = np.where(R <= r_max)
        R = R[Xr, Yr]
        V = img[Xr, Yr]

    if kwargs.get("normalize", False):
        V = V - np.min(V)
        V = V / np.max(V)

    return [R.flatten(), V.flatten()]

In [ ]:
# polar radius-only image at initial guess (8,7)

# reasonable radius of the beads in the image
r_max = 2.0

# initial guess
P0 = P0_global - cropbox_offset

# radius and value vectors
R, V_true = reduce_img_to_polar_radius(img_raw, *P0, r_max=r_max, normalize=True)

plt.figure(figsize=(14,6))

plt.subplot(121)
plt.title("Polar radius-only image at {}, r_max={:.2f}".format(P0, r_max))

plt.scatter(R, V_true, label="normalised pixel values")

plt.xlabel("Radius")
plt.ylabel("Pixel value")
plt.legend()
plt.grid()

plt.subplot(122)
plt.title("Image")

plt.imshow(img_raw)
plt.colorbar().set_label("pixel value")

plt.plot([P0[0]] * 2, [0, img.shape[0]-1], c="C1", ls="--")  # vertical
plt.plot([0, img.shape[1]-1], [P0[1]] * 2, c="C1", ls="--")  # horizontal

plt.show()

### Find best sigmoid fit

Find best fit of the sigmoid function ($f(x) = \frac{e^x}{e^x+1}$) to the data in the plot. The code for best sinc function ($f(x) = sin(x) x^{-1}$) is also available below.

Please note: the parameters are found using a brounded gradient descent algorithm. Useful bounds have been identified using a trial-and-error approach.

The exact sigmoid function for the fit is $f(x) = \frac{\beta_0}{e^{\beta_1 x - 0.5 \beta_1 r_{max}}}$. $\beta_0$ corresponds to the amplitude, $\beta_1$ corresponds to the slope, and the expression $\beta_1 x - 0.5 \beta_1 r_{max}$ moves the steepest part of the slope to $x = 0.5 r_{max}$.

Lastly, we plot the best sigmoid fit onto the data.

In [ ]:
#%%timeit

#global model_sinc, res

def model_sinc(R, V_true, full_output=False):
    """
    Find parametes for the sinc model by minimizing MSE from model to observation

    Parameters
    ----------
    full_output: boolean
        If true all output from scipy.optimize.minimize is returned, if false only OptimizationResult.x is returned

    Returns
    -------
    List of optimal parameters or full OptimizationResult object depending on method parameters
    """
    min_amplitude = V_true.max()
    if min_amplitude < 17000.0:
        min_amplitude = 17000.0
    amplitude_range = [
        min_amplitude, 
        1.2 * min_amplitude
    ]
    period_range = [4.0, 8.0]
    res = minimize(
        lambda P_inner, R_inner, V_true_inner: metrics.mean_squared_error(
            V_true_inner,
            P_inner[0] * np.sinc(R_inner / P_inner[1])
        ),
        (
            np.mean(amplitude_range),
            np.mean(period_range)
        ),
        args=(
            R, V_true,
        ),
        bounds=(
            amplitude_range,
            period_range
        )
    )
    if full_output:
        return res
    return res.x

def model_sigmoid(R: np.ndarray, V_true: np.ndarray, r_max: float, full_output=False):
    """
    Find parametes for the sigmoid model by minimizing MSE from model to observation

    Parameters
    ----------
    full_output: boolean
        If true all output from scipy.optimize.minimize is returned, if false only OptimizationResult.x is returned

    Returns
    -------
    List of optimal parameters or full OptimizationResult object depending on method parameters
    """
    
    amplitude_range = [
        1.0, 
        1.5
    ]
    slope_range = [0.0, 3.0]
    
    res = minimize(
        lambda P_inner, R_inner, V_true_inner, r_max_inner: metrics.mean_squared_error(
            V_true_inner,
            P_inner[0] / (
                1 + np.exp(
                    P_inner[1] * R_inner - (P_inner[1] * r_max_inner / 2)
                )
            )
        ),
        (
            np.mean(amplitude_range),
            np.mean(slope_range)
        ),
        args=(
            R, V_true, r_max,
        ),
        bounds=(
            amplitude_range,
            slope_range
        )
    )
    if full_output:
        return res
    return res.x

#res = model_sinc(R, V_true, full_output=True)
res = model_sigmoid(R, V_true, r_max, full_output=True)
res

In [ ]:
R_pred = np.linspace(0, r_max, 100)
#V_pred = res.x[0] * np.sinc(R_pred / res.x[1])  # sin(x)/x
V_pred = res.x[0] / (1 + np.exp(res.x[1] * R_pred - (res.x[1] * r_max / 2)))  # sigmoid function

plt.figure(figsize=(14,6))

plt.subplot(121)
plt.title("Polar radius-only image at {}, r_max={}".format(P0, r_max))
plt.xlabel("Radius")
plt.ylabel("Pixel value")

plt.scatter(R, V_true, label="pixel values")
#plt.plot(R_pred, V_pred, c="C1", label="sin(x)/x, MSE={:.2e}".format(res.fun))
plt.plot(R_pred, V_pred, c="C1", label="sigmoid, MSE={:.2e}".format(res.fun))
plt.legend()
plt.grid()

plt.subplot(122)
plt.title("Image")

plt.imshow(img_raw)
plt.colorbar().set_label("pixel value")

plt.plot([P0[0]] * 2, [0, img_raw.shape[0]-1], c="C1", ls="--")  # vertical
plt.plot([0, img_raw.shape[1]-1], [P0[1]] * 2, c="C1", ls="--")  # horizontal

plt.show()

### Investigating the cost function

We investigate the cost function at every combination of $(x,y)$ in a sensibly dense grid to find a suitable method and bounds for minimization. We generate a cost map at resolution 20/pixel.

Each cost function caps the cost a a given value. This is done because there is a border of local maxima around the global minimum at the peak, and the algorithm may be confused by this.

In [ ]:
def sinc_cost_at_xy(P, img_window, r_max: float):
    """
    Cost function for peak P at image img_window analysed with a radius of r_max
    
    Parameters
    ----------
    P: list[int]
        Coordinates, (x,y)
    img_window: ndarray
        Numpy array representing the image. img_window[y,x] as usual for numpy row-major notation.
    r_max: float
        The distance to which we crop the radius image
    """
    
    assert len(P) == 2, "Please provide exactly 2 coordinates"
    assert img_window.ndim == 2, "Please provide img_window with two dimensions"
    
    """
    Define an optinal maximum cost. When approaching the target, it is observed that cost
    can increase rapidly before decreasing to the global minimum. By defining a
    maximum cost, this "cost barrier" is eliminated. Only side-effect: we need to
    get close to the global minimum before we get a gradient for the minimizer.
    
    This value has been observed to cap cost except exactly at the target.
    """
    max_cost = 7e6

    # if pixel value is less than what we expect from a peak, just quit here
    #if img_window[int(round(P[1])), int(round(P[0]))] < 20000.0:
    #    return max_cost
    
    R, V_true = reduce_img_to_polar_radius(img_window, P[0], P[1], r_max)
    res = model_sinc(R, V_true, full_output=True)
    
    if res.fun > max_cost:
        return max_cost
    return res.fun


def sigmoid_cost_at_xy(P, img_window, r_max: float):
    """
    Cost function for peak P at image img_window analysed with a radius of r_max
    
    Parameters
    ----------
    P: list[int]
        Coordinates, (x,y)
    img_window: ndarray
        Numpy array representing the image. img_window[y,x] as usual for numpy row-major notation.
    r_max: float
        The distance to which we crop the radius image
    """
    
    assert len(P) == 2, "Please provide exactly 2 coordinates"
    assert img_window.ndim == 2, "Please provide img_window with two dimensions"
    
    """
    Define an optinal maximum cost. When approaching the target, it is observed that cost
    can increase rapidly before decreasing to the global minimum. By defining a
    maximum cost, this "cost barrier" is eliminated. Only side-effect: we need to
    get close to the global minimum before we get a gradient for the minimizer.
    
    This value has been observed to cap cost except exactly at the target.
    """
    max_cost = 0.05  # 0.05 for r_max = 2.0, 0.03 for r_max = 3.0

    # if pixel value is less than what we expect from a peak, just quit here
    #if img_window[int(round(P[1])), int(round(P[0]))] < 20000.0:
    #    return max_cost
    
    R, V_true = reduce_img_to_polar_radius(img_window, P[0], P[1], r_max=r_max, normalize=True)
    res = model_sigmoid(R, V_true, r_max, full_output=True)
    
    if res.fun > max_cost:
        return max_cost
    return res.fun

In [ ]:
# make a high-resolution cost map
# density is number of evaluations per pixel per axis
cost_map_density = 5

X, Y = np.meshgrid(
    np.linspace(0, img_raw.shape[1], num=cost_map_density*img_raw.shape[1]) - 0.5,
    np.linspace(0, img_raw.shape[0], num=cost_map_density*img_raw.shape[0]) - 0.5
)
cost_map = pd.DataFrame({
    "x": X.flatten(),
    "y": Y.flatten()
})
cost_map.shape

In [ ]:
%%timeit -n 1 -r 1

# fill cost map

global cost_map, cost_img
def apply_cost(row):
    return sigmoid_cost_at_xy(row[["x", "y"]].tolist(), img_raw, r_max)

cost_map["cost"] = cost_map.apply(apply_cost, axis=1)
cost_img = cost_map["cost"].values.reshape(np.array(img.shape)*cost_map_density)

In [ ]:
plt.figure(figsize=(16,8))
plt.title("Capped cost map")

#plt.imshow(cost_img.clip(max=0.05))
plt.imshow(cost_img)
plt.colorbar().set_label("cost [MSE]")

xtick_labels = np.arange(img_raw.shape[1])
xtick_pos = (xtick_labels + 0.5) * cost_map_density - 0.5 # position needs to be moved at both unscaled and scaled axis
plt.xticks(xtick_pos, xtick_labels)

ytick_labels = np.arange(img_raw.shape[0])
ytick_pos = (ytick_labels + 0.5) * cost_map_density - 0.5
plt.yticks(ytick_pos, ytick_labels)
plt.grid(linestyle="dotted")

plt.show()

From the cost-map above, we see that within at least 4 pixel of the peak there is no other local minimum. If we can get close enough to the peak to avoid other local minima, and if we can search the area at points dense enough to find the peak, we should be able to find tune the peak using a gradient descent algorithm.

Below we see the application of a grid search algorithm that searches each corner of every pixel. It uses a gradient descent to fine tune the result.

In [ ]:
P0 = (10, 9)
P0_margin = 2  # notice that we do (P0_margin * 4 + 1) ** 2 evaluations of the cost function
res = brute(
    sigmoid_cost_at_xy,
    (
        (P0[0] - P0_margin, P0[0] + P0_margin),
        (P0[1] - P0_margin, P0[1] + P0_margin)
    ),
    args=(img_raw, r_max,),
    Ns=P0_margin*2+1
)
res

In [ ]:
# radius-only polar image and values
R, V_true = reduce_img_to_polar_radius(img_raw, *res, r_max=r_max, normalize=True)

plt.figure(figsize=(14,6))
plt.subplot(121)
plt.title("Pixel values and best sin(x)/x fit")

# calculate predicted values and plot along with true values
R_pred = np.linspace(0, r_max, 100)

#amplitude, period = model_sinc(R, V_true)
#V_pred = amplitude * np.sinc(R_pred / period)  # sin(x)/x
#r2 = metrics.r2_score(V_true, amplitude * np.sinc(R / period))

amplitude, slope = model_sigmoid(R, V_true, r_max)
V_pred = amplitude / (1 + np.exp(slope * R_pred - (slope * r_max / 2)))
r2 = metrics.r2_score(V_true, amplitude / (1 + np.exp(slope * R - (slope * r_max / 2))))

plt.scatter(R, V_true, label="pixel values (n={})".format(V_true.shape[0]))
plt.plot(R_pred, V_pred, c="C1", label="sin(x)/x, r²={:.3f}".format(r2))

plt.legend()
plt.grid()

plt.subplot(122)
plt.title("Image with detected peak at ({:.2f}, {:.2f})".format(*res))

plt.imshow(img_raw)
plt.colorbar().set_label("pixel value")

# out sampling area
circle = patches.Circle(
    res,
    radius=r_max,
    edgecolor="C5",
    fill=False,
    label="initial simplex"
)
plt.gca().add_patch(circle)

plt.plot([res[0]] * 2, [0, img_raw.shape[0]-1], c="C1", ls="--")  # vertical
plt.plot([0, img_raw.shape[1]-1], [res[1]] * 2, c="C1", ls="--")  # horizontal
#plt.legend()

plt.show()

### Conclusion to fine-tuning

Above we see that this approach works well under the criteria it is tested under in this notebook. This approach is the currently implemented approach in the code base.

**Some notes from production testing**:

* This approach works best with $r_{max} \approx 2$ and with normalised $V$.
* If the peak is exactly at the corner of a pixel (e.g. where four pixels join up), this algorithm fails.